In [1]:
import pandas as pd
from rouge import Rouge

In [2]:
dataFull=pd.read_csv("Reviews.csv")
dataFull.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Score','Time'],axis=1,inplace=True)
#dataFull.drop_duplicates(subset=['Text'],inplace=True)#dropping duplicates
dataFull.dropna(axis=0,inplace=True)#dropping na

In [3]:
dataFull.shape

(568427, 2)

In [4]:
dataFull.tail()

,Summary,Text
568449,Will not do without,Great for sesame chicken..this is a good if no...
568450,disappointed,I'm disappointed with the flavor. The chocolat...
568451,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,Favorite Training and reward treat,These are the BEST treats for training and rew...
568453,Great Honey,"I am very satisfied ,product is as advertised,..."


In [5]:
df=dataFull.tail(100000)

In [6]:
t1=df["Text"][468451]
t1

"Until my wife and I started a weight loss effort earlier this summer, we hadn't eaten all that much Progresso soup. Because of their variety of flavors and low calories, we decided to give them a try, and I'm very glad we did.<br /><br />This soup is only 60 calories per serving, but has great flavor, as do all the Progresso soups we've tried. Many lo-cal soups are just blah tasting, but Progresso puts a nice blend of spices into everything we've tried, and this soup is no exception. I'm quite impressed and heartily recommend this variety to dieters who'd like a low calorie meal that they can enjoy the taste of!"

In [7]:
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict




def summarize(text,n):
    sents=sent_tokenize(text)
    
    for i in range(len(sents)):
        row=sents[i]
        row = row.replace("\n", "")
        row = row.replace("\t", "")
        row = row.replace("-", "")
        sents[i] = row 
    
           
    assert n<=len(sents)
    word_sent=word_tokenize(text.lower())
    freq=FreqDist(word_sent)
    nlargest(20,freq,key=freq.get)
    customStopwords=set(stopwords.words('english')+list(punctuation)+list("\n"))
    word_sent=[word for word in word_sent if word not in customStopwords]
    
    print(len(sents))
    
    ranking=defaultdict(int)
    for i,sent in enumerate(sents):
        for w in word_tokenize(sent.lower()):
            if w in freq:
                ranking[i]+=freq[w]
    sents_idx=nlargest(n,ranking,key=ranking.get)
    return [sents[j] for j in sorted(sents_idx)]
    

summarize(t1,1)

#Analyze frequency of words and see effect of rare words

4


["Because of their variety of flavors and low calories, we decided to give them a try, and I'm very glad we did.<br /><br />This soup is only 60 calories per serving, but has great flavor, as do all the Progresso soups we've tried."]

In [8]:
from sumy.summarizers import luhn
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.nlp.tokenizers import Tokenizer
from gensim.summarization import summarize
from sumy.summarizers.luhn import LuhnSummarizer 
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer

LANGUAGE = "english"
SENTENCES_COUNT = 1

#text=t1

#parser = PlaintextParser.from_string((text), Tokenizer(LANGUAGE))
stemmer = Stemmer(LANGUAGE)




def lexrank_summarizer(text):
    parser = PlaintextParser.from_string((text), Tokenizer(LANGUAGE))
    summarizer_LexRank = LexRankSummarizer(stemmer)
    summarizer_LexRank.stop_words = get_stop_words(LANGUAGE)
    output=''
    for sentence in summarizer_LexRank(parser.document, SENTENCES_COUNT):
        output+=str(sentence)
    return output
        
        
def lsa_summarizer(text):
    parser = PlaintextParser.from_string((text), Tokenizer(LANGUAGE))
    summarizer_lsa = Summarizer(stemmer)
    summarizer_lsa.stop_words = get_stop_words(LANGUAGE)
    ab=""
    for sentence in summarizer_lsa(parser.document, SENTENCES_COUNT):
        #print (sentence)
        #print(type(sentence))
        ab+=str(sentence)
    return ab
   # return sentence
def luhn_summarizer(text):
    parser = PlaintextParser.from_string((text), Tokenizer(LANGUAGE))
    summarizer_luhn = LuhnSummarizer(stemmer)
    summarizer_luhn.stop_words = get_stop_words(LANGUAGE)
    abc=''
    for sentence in summarizer_luhn(parser.document, SENTENCES_COUNT):
        abc+=str(sentence)
    return abc
        
def gensim_summarizer():
    print(summarize(text))

print("Lexranks Summary:")
print(lexrank_summarizer(t1))
print("\n")
print("LSA Summary:")
print(lsa_summarizer(t1))
print("\n")
print("Luhn's Summary:")
print(luhn_summarizer(t1))
#print("========")
#gensim_summarizer()

C:\Users\ishan\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Lexranks Summary:
Until my wife and I started a weight loss effort earlier this summer, we hadn't eaten all that much Progresso soup.


LSA Summary:
Until my wife and I started a weight loss effort earlier this summer, we hadn't eaten all that much Progresso soup.


Luhn's Summary:
Because of their variety of flavors and low calories, we decided to give them a try, and I'm very glad we did.<br /><br />This soup is only 60 calories per serving, but has great flavor, as do all the Progresso soups we've tried.


In [11]:
r1=0.0
r2=0.0
rl=0.0
ce=0
rouge=Rouge()
for i in range(468450,568454):
    #print(df["Summary"][i])
    try:
        sc=rouge.get_scores(lexrank_summarizer(df["Text"][i]),df["Summary"][i])
    except:
        ce+=1
        continue
    r1+=sc[0]['rouge-1']['f']
    r2+=sc[0]['rouge-2']['f']
    rl+=sc[0]['rouge-l']['f']
print(ce)

492


In [18]:
print(r1/99508)
print(r2/99508)
print(rl/99508)

0.06599310420831273
0.01720169629091797
0.04830501133640649


In [9]:
rr1=0.0
rr2=0.0
rrl=0.0
c=0
rouge2=Rouge()
for i in range(468450,568454):
    #print(df["Summary"][i])
    try:
        sco=rouge2.get_scores(lsa_summarizer(df["Text"][i]),df["Summary"][i])
    except:
        c+=1
        continue
    rr1+=sco[0]['rouge-1']['f']
    rr2+=sco[0]['rouge-2']['f']
    rrl+=sco[0]['rouge-l']['f']
print(c)

C:\Users\ishan\Anaconda3\lib\site-packages\sumy\summarizers\lsa.py:76: UserWarning: Number of words (3) is lower than number of sentences (4). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))
C:\Users\ishan\Anaconda3\lib\site-packages\sumy\summarizers\lsa.py:76: UserWarning: Number of words (4) is lower than number of sentences (5). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))
C:\Users\ishan\Anaconda3\lib\site-packages\sumy\summarizers\lsa.py:76: UserWarning: Number of words (6) is lower than number of sentences (7). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))
C:\Users\ishan\Anaconda3\lib\site-packages\sumy\summarizers\lsa.py:76: UserWarning: Number of words (5) is lower than number of sentences (8). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))
C:\Users\ishan\Anaconda3\lib\site-packages\sumy\summarizers\lsa.py:76: UserWarning: 

493


In [10]:
print(rr1/99507)
print(rr2/99507)
print(rrl/99507)

0.04974007379017464
0.010007397639722198
0.03342907689476802


In [13]:
rrr1=0.0
rrr2=0.0
rrrl=0.0
rouge3=Rouge()
count=0
for i in range(468450,568454):
    #print(df["Summary"][i])
    try:
        scor=rouge3.get_scores(luhn_summarizer(df["Text"][i]),df["Summary"][i])
    except:
        count+=1
        continue
    rrr1+=scor[0]['rouge-1']['f']
    rrr2+=scor[0]['rouge-2']['f']
    rrrl+=scor[0]['rouge-l']['f']

In [14]:
print(count)

494


In [16]:
print(rrr1/99506)
print(rrr2/99506)
print(rrrl/99506)

0.06166717654986019
0.015335679932927442
0.04407498644017556


In [35]:
o1=0.0
o2=0.0
ol=0.0
cou=0
rouge4=Rouge()
for i in range(468450,568454):
    #print(i)
    tot=""
    try:
        for s in summarize(df["Text"][i],1):
            tot+=str(s)
    #print(tot)
        score=rouge3.get_scores(tot,df["Summary"][i])
    except:
        cou+=1
        continue
    o1+=score[0]['rouge-1']['f']
    o2+=score[0]['rouge-2']['f']
    ol+=score[0]['rouge-l']['f']
print(cou)

34229


In [36]:
print(o1/99988)
print(o2/99988)
print(ol/99988)

0.028903905626081347
0.0057598653759092355
0.015804811527395057


In [17]:
print(100000/568454)

0.17591572932902222
